In [88]:
import pandas as pd
import pyodbc


In [89]:
# spring.datasource.url=jdbc:sqlserver://localhost:1433;databaseName=IMDb_spring;encrypt=true;trustServerCertificate=true;

driver = "ODBC Driver 18 for SQL Server"
server = "localhost"
database = "IMDb_spring"
username = "sa"
password = "Password123"
run = False


In [90]:
connection_string = f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes"
conn = None
if run:
    conn = pyodbc.connect(connection_string)

row_limit = 3000


In [91]:
def return_null_if_empty(value):
    if value == "" or value == "\\N":
        return "null"
    else:
        return value

def return_null_if_empty_and_stringify(value):
    if value == "" or value == "\\N":
        return "null"
    else:
        return f"'{value}'"


def create_or_find_title_type(type_name):
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM title_types WHERE name = '{type_name}'")
    row = cursor.fetchone()
    if row is None:
        cursor.execute(f"INSERT INTO title_types (title_type_id, name) values (next value for title_types_id_seq,'{type_name}')")
        conn.commit()
        cursor.execute(f"SELECT * FROM title_types WHERE name = '{type_name}'")
        row = cursor.fetchone()
    return row[0]

def create_or_find_genre(genre_name):
    try:
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM genres WHERE name = '{genre_name}'")
        row = cursor.fetchone()
        if row is None:
            cursor.execute(f"INSERT INTO genres (genre_id, name) values (next value for genres_id_seq,'{genre_name}')")
            conn.commit()
            cursor.execute(f"SELECT * FROM genres WHERE name = '{genre_name}'")
            row = cursor.fetchone()
        return row[0]
    except Exception as e:
        print(e)

def add_genres_to_title_basics(tconst, genres_list):
    genres = genres_list.split(",")
    for genre in genres:
        try:
            genre_id = create_or_find_genre(genre)
            cursor = conn.cursor()
            cursor.execute(f"INSERT INTO title_genres (tconst, genre_id) values ('{tconst}', {genre_id})")
        except Exception as e:
            print(e)
    conn.commit()


def insert_title_basics():
    df = pd.read_csv("title.basics.tsv", sep="\t", nrows=row_limit)
    df = df.where(pd.notnull(df), None)
    for index, row in df.iterrows():
        try:
            tconst = row["tconst"]
            title_type = create_or_find_title_type(row["titleType"])
            primary_title = "{0}".format(row["primaryTitle"].replace("'", "''"))
            original_title = "{0}".format(row["originalTitle"].replace("'", "''"))
            is_adult = row["isAdult"]
            start_year = row["startYear"]
            end_year = return_null_if_empty(row["endYear"])
            runtime_minutes = return_null_if_empty(row["runtimeMinutes"])
            genres = return_null_if_empty(row["genres"])
            cursor = conn.cursor()
            cursor.execute(f"INSERT INTO title_basics (tconst, title_type_id, primary_title, original_title, is_adult, start_year, end_year, runtime_minutes) values ('{tconst}', {title_type}, '{primary_title}', '{original_title}', {is_adult}, {start_year}, {end_year}, {runtime_minutes})")
            if genres is not None:
                add_genres_to_title_basics(tconst, genres)
        except Exception as e:
            print(e)
    conn.commit()


In [92]:
def create_or_find_title_akas_type(type_name):
    if type_name is None or type_name == "" or type_name == "\\N" or type_name == "null":
        return "null"
    try:
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM akas_types WHERE name = '{type_name}'")
        row = cursor.fetchone()
        if row is None:
            cursor.execute(f"INSERT INTO akas_types (akas_type_id, name) values (next value for IMDb_spring.dbo.akas_types_id_seq,'{type_name}')")
            conn.commit()
            cursor.execute(f"SELECT * FROM akas_types WHERE name = '{type_name}'")
            row = cursor.fetchone()
        return row[0]
    except Exception as e:
        print(e)


def insert_title_akas():
    df = pd.read_csv("title.akas.tsv", sep="\t", nrows=row_limit)
    df = df.where(pd.notnull(df), None)
    for index, row in df.iterrows():
        try:
            title_id = row["titleId"]
            ordering = row["ordering"]
            title = "{0}".format(row["title"].replace("'", "''"))
            region = return_null_if_empty_and_stringify(row["region"])
            language = return_null_if_empty_and_stringify(row["language"])
            types = return_null_if_empty(row["types"])
            type_id = create_or_find_title_akas_type(types)
            attributes = return_null_if_empty_and_stringify(row["attributes"])
            is_original_title = row["isOriginalTitle"]
            cursor = conn.cursor()
            cursor.execute(f"INSERT INTO title_akas (tconst, ordering, akas_type_id, title, region, language, attributes, is_original_title)  values ('{title_id}', {ordering}, {type_id}, '{title}', {region}, {language}, {attributes}, {is_original_title})")
        except Exception as e:
            print(e)

    conn.commit()

'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribute 'cursor'
'NoneType' object has no attribu

AttributeError: 'NoneType' object has no attribute 'commit'

In [ ]:
def create_or_find_professions(type_name):
    if type_name is None or type_name == "" or type_name == "\\N" or type_name == "null":
        return "null"
    try:
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM professions WHERE name = '{type_name}'")
        row = cursor.fetchone()
        if row is None:
            cursor.execute(f"INSERT INTO professions (profession_id, name) values (next value for IMDb_spring.dbo.professions_id_seq,'{type_name}')")
            conn.commit()
            cursor.execute(f"SELECT * FROM professions WHERE name = '{type_name}'")
            row = cursor.fetchone()
        return row[0]
    except Exception as e:
        print(e)

def add_professions_to_name_basics(nconst, professions_list):
    professions = professions_list.split(",")
    for profession in professions:
        try:
            profession_id = create_or_find_professions(profession)
            cursor = conn.cursor()
            cursor.execute(f"INSERT INTO name_professions (nconst, profession_id) values ('{nconst}', {profession_id})")
        except Exception as e:
            print(e)
    conn.commit()

def add_name_known_for_to_name_basics(nconst, known_for_titles_list):
    known_for_titles = known_for_titles_list.split(",")
    for known_for_title in known_for_titles:
        try:
            cursor = conn.cursor()
            cursor.execute(f"INSERT INTO name_known_for_titles (nconst, tconst) values ('{nconst}', '{known_for_title}')")
        except Exception as e:
            print(e)
    conn.commit()


def insert_name_basics():
    df = pd.read_csv("name.basis.tsv", sep="\t", nrows=row_limit)
    df = df.where(pd.notnull(df), None)
    for index, row in df.iterrows():
        try:
            nconst = row["nconst"]
            primary_name = "{0}".format(row["primaryName"].replace("'", "''"))
            birth_year = return_null_if_empty(row["birthYear"])
            death_year = return_null_if_empty(row["deathYear"])
            primary_profession = return_null_if_empty(row["primaryProfession"])
            known_for_titles = return_null_if_empty(row["knownForTitles"])
            cursor = conn.cursor()
            cursor.execute(f"INSERT INTO name_basics (nconst, primary_name, birth_year, death_year) values ('{nconst}', '{primary_name}', {birth_year}, {death_year})")
            if primary_profession is not None:
                add_professions_to_name_basics(nconst, primary_profession)
            if known_for_titles is not None:
                add_name_known_for_to_name_basics(nconst, known_for_titles)
        except Exception as e:
            print(e)
    conn.commit()


In [ ]:
def add_writers_to_titles(tconst, writers_list):
    writers = writers_list.split(",")
    for writer in writers:
        try:
            cursor = conn.cursor()
            cursor.execute(f"INSERT INTO title_writers (tconst, nconst) values ('{tconst}', '{writer}')")
        except Exception as e:
            print(e)
    conn.commit()

def add_directors_to_titles(tconst, directors_list):
    directors = directors_list.split(",")
    for director in directors:
        try:
            cursor = conn.cursor()
            cursor.execute(f"INSERT INTO title_directors (tconst, nconst) values ('{tconst}', '{director}')")
        except Exception as e:
            print(e)
    conn.commit()

def insert_title_crew():
    df = pd.read_csv("title.crew.tsv", sep="\t", nrows=row_limit)
    df = df.where(pd.notnull(df), None)
    for index, row in df.iterrows():
        try:
            tconst = row["tconst"]
            directors = return_null_if_empty(row["directors"])
            writers = return_null_if_empty(row["writers"])
            if directors is not None:
                add_directors_to_titles(tconst, directors)
            if writers is not None:
                add_writers_to_titles(tconst, writers)
        except Exception as e:
            print(e)
    conn.commit()


In [ ]:
def create_or_find_principal_category(type_name):
    if type_name is None or type_name == "" or type_name == "\\N" or type_name == "null":
        return "null"
    try:
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM principal_category WHERE name = '{type_name}'")
        row = cursor.fetchone()
        if row is None:
            cursor.execute(f"INSERT INTO principal_category (principal_category_id, name) values (next value for IMDb_spring.dbo.principal_category_id_seq,'{type_name}')")
            conn.commit()
            cursor.execute(f"SELECT * FROM principal_category WHERE name = '{type_name}'")
            row = cursor.fetchone()
        return row[0]
    except Exception as e:
        print(e)

def insert_title_principals():
    df = pd.read_csv("title.principals.tsv", sep="\t", nrows=row_limit)
    df = df.where(pd.notnull(df), None)
    for index, row in  df.iterrows():
        try:
            tconst = row["tconst"]
            ordering = row["ordering"]
            nconst = row["nconst"]
            category = return_null_if_empty(row["category"])
            job = return_null_if_empty(row["job"])
            characters = return_null_if_empty(row["characters"])
            characters = characters.replace("[", "")
            characters = characters.replace("]", "")
            characters = characters.replace("'", "")
            characters = characters.replace('"', "")
            principal_category_id = create_or_find_principal_category(category)
            cursor = conn.cursor()
            cursor.execute(f"INSERT INTO title_principals (tconst, ordering, nconst, category_id, job, characters) values ('{tconst}', {ordering}, '{nconst}', {principal_category_id}, '{job}', '{characters}')")
        except Exception as e:
            print(e)
    conn.commit()


In [ ]:
if conn is not None:
    insert_title_basics()
    insert_title_akas()
    insert_name_basics()
    insert_title_crew()
    insert_title_principals()